In [ ]:
# Import necessary libraries
import os  # Provides functions to interact with the operating system, like accessing environment variables
import time  # Used for time-related tasks, such as delays or measuring durations
import google.generativeai as genai  # Import the Google Generative AI library for using the generative model

# Configure the Generative AI model by providing the API key from environment variables
genai.configure(api_key=os.environ["GEMINI_API_KEY"])  # Retrieve the API key from the environment variable 'GEMINI_API_KEY'

In [ ]:
# Function to upload a file to Gemini
def upload_to_gemini(path, mime_type=None):
    # Upload the file to Gemini using the provided file path and optional MIME type
    file = genai.upload_file(path, mime_type=mime_type)
    
    # Print the display name of the uploaded file and its URI (Uniform Resource Identifier)
    print(f"Uploaded file '{file.display_name}' as: {file.uri}")
    
    # Return the uploaded file object, which contains information like the file's URI
    return file

In [ ]:
# Function to wait for files to be processed and become active
def wait_for_files_active(files):
    # Print a message indicating the start of the file processing wait
    print("Waiting for file processing...")
    
    # Iterate over the list of files and check the processing status for each one
    for name in (file.name for file in files):  # Loop through each file's name in the files list
        # Retrieve the current state of the file using the file name
        file = genai.get_file(name)
        
        # While the file is still being processed, print a dot and wait for 10 seconds
        while file.state.name == "PROCESSING":
            print(".", end="", flush=True)  # Print a dot without a newline to indicate waiting
            time.sleep(10)  # Wait for 10 seconds before checking the file status again
            file = genai.get_file(name)  # Re-check the file status after the wait
        
        # If the file is not in the 'ACTIVE' state after processing, raise an error
        if file.state.name != "ACTIVE":
            raise Exception(f"File {file.name} failed to process")  # Raise an exception if the file is not active
    
    # Print a 


In [ ]:
# Configuration settings for text generation
generation_config = {
  "temperature": 0.2,  # Controls the randomness of the output. Lower values make the output more predictable.
  "top_p": 0.95,       # Nucleus sampling parameter; the model considers tokens whose cumulative probability mass is within this value.
  "top_k": 40,         # Limits the model to consider only the top k tokens during generation.
  "max_output_tokens": 8192,  # Sets the maximum number of tokens (words or symbols) that can be generated in the response.
  "response_mime_type": "text/plain",  # Specifies the format of the response (plain text).
}

# Create a GenerativeModel instance with the specified model name and generation configuration
model = genai.GenerativeModel(
  model_name="gemini-1.5-pro-002",  # Model version used for text generation
  generation_config=generation_config,  # Pass in the configuration settings
  system_instruction="Answer Question for the PDF Attachment only",  # System instruction to ensure the model answers based only on the provided PDF files
)

# Upload the PDF files to Gemini and store them in the 'files' list
files = [
  upload_to_gemini("D:/data science/projects/my projects/Health Summary Generator/Media/sample_input_part_1.pdf", mime_type="application/pdf"),  # Upload first part of the document
  upload_to_gemini("D:/data science/projects/my projects/Health Summary Generator/Media/sample_input_part_2.pdf", mime_type="application/pdf"),  # Upload second part
  upload_to_gemini("D:/data science/projects/my projects/Health Summary Generator/Media/sample_input_part_3.pdf", mime_type="application/pdf"),  # Upload third part
]

# Some files have a processing delay. Wait for them to be processed and become active.
wait_for_files_active(files)  # Call the function to check the status of the uploaded files and wait until they are active


Uploaded file 'sample_input_part_1.pdf' as: https://generativelanguage.googleapis.com/v1beta/files/uoxq4mdgoii3
Uploaded file 'sample_input_part_2.pdf' as: https://generativelanguage.googleapis.com/v1beta/files/fvug7md7fhfr
Uploaded file 'sample_input_part_3.pdf' as: https://generativelanguage.googleapis.com/v1beta/files/mdpi57mmdp69
Waiting for file processing...
...all files ready



In [ ]:
# Start a chat session with the model, providing chat history and initial instructions
chat_session = model.start_chat(
  history=[  # The conversation history is passed as a list of message objects
    {
      "role": "user",  # The role of the participant (in this case, the user)
      "parts": [  # The parts of the user's message (i.e., content being sent)
        files[0],  # Attach the first uploaded file (sample_input_part_1.pdf)
        files[1],  # Attach the second uploaded file (sample_input_part_2.pdf)
        files[2],  # Attach the third uploaded file (sample_input_part_3.pdf)
        "Tell Client information name, phone number, address, email address detail",  # User's query asking for specific client information from the PDF files
      ],
    },
    {
      "role": "model",
      "parts": [
        "The client's name is Teleah Grand.  The phone number is 214-563-1688. The address is 2200 Victory Ave #702, Dallas, TX 75219.  No email address is listed.",
      ],
    },
    {
      "role": "user",
      "parts": [
        "Tell Patient information Name, Breed, DOB or Age, Gender, Microship number",
      ],
    },
    {
      "role": "model",
      "parts": [
        "The patient's name is Buckshot. He is a French Bulldog. His date of birth is February 28, 2018 (making him 6 years old at the time of this record). He is a neutered male. His microchip number is 981020021462501.",
      ],
    },
    {
      "role": "user",
      "parts": [
        "Tell Veterinary Clinic Information as Visit Data, Clinic Name, Phone number",
      ],
    },
    {
      "role": "model",
      "parts": [
        "The veterinary clinic is **Animal Care Center at Stonebridge Ranch**.  Their phone number is 214-563-1688.  The provided records span from May 5, 2018, to October 14, 2024.  It is not possible to isolate visit dates for all appointments from this document alone.  Each page of the record indicates the date it was generated, which may or may not correspond to an appointment date.  Specific appointment dates are sometimes, but not always, included in the record notes.\n",
      ],
    },
    {
      "role": "user",
      "parts": [
        "Tell about Vaccinations information such as Vaccine, Date, Manufacturer tag number",
      ],
    },
    {
      "role": "model",
      "parts": [
        "The following vaccinations are recorded for Buckshot:\n\n* **Rabies:** June 1, 2018. Merial (Tag #432943).  Expiration: June 22, 2019.  (Note: A rabies *titer* was performed on November 30, 2020, with results pending.)\n* **DA2PP (Distemper, Adenovirus, Parainfluenza, Parvovirus):**\n    * May 13, 2018 (Pfizer)\n    * June 1, 2018 (Pfizer)\n    * June 21, 2018 (Pfizer)\n* **Bordetella (Kennel Cough):**\n    * May 11, 2018 (Intervet)\n    * June 9, 2022 (Intervet. Vaccine #04080252. Expires 10/14/2024)\n    * October 14, 2021 (Intervet. Vaccine #04080214. Expires 10/27/2023)\n    * April 16, 2021 (Intervet. Vaccine #04080195. Expires 05/19/2023)\n    * November 11, 2020 (Intervet. Vaccine #02001135B. Expires 11/15/2022)\n    * October 11, 2019 (Intervet. Vaccine #02001131B. Expires 12/10/2021)\n    * April 26, 2019 (Intervet. Vaccine #04080116. Expires 5/23/2020)\n    * November 5, 2018 (Intervet. Vaccine #04080116. Expires 5/23/2020)\n    * January 23, 2023 (Intervet. Vaccine #0408276. Expires 03/15/2025)\n\nIt's important to note that this information is based on the provided document and may not be a complete vaccination history.  There are notes indicating other vaccinations may have been given, but the details are not present in this particular document.\n",
      ],
    },
    {
      "role": "user",
      "parts": [
        "Tell about medical information as Marker wieght for date 1 to date 5",
      ],
    },
    {
      "role": "model",
      "parts": [
        "Buckshot's weight fluctuated over the years documented in the records. Here are five specific weight measurements:\n\n1. **February 28, 2023:** 21.50 lb\n2. **May 3, 2023:** 20.40 lb\n3. **May 8, 2024:** 25.50 lb\n4. **May 8, 2024 (same day, later entry):** 25.5 lb\n5. **September 12, 2024:** 26.00 lb\n\nIt's worth noting that Buckshot's weight seems to have increased significantly between May 2023 and May 2024.  The records indicate numerous instances of gastrointestinal issues, so fluctuations in weight could be related to those problems.  Always consult with a veterinarian to interpret medical records and address any concerns about a pet's health or weight.\n",
      ],
    },
    {
      "role": "user",
      "parts": [
        "Patient alters detial",
      ],
    },
    {
      "role": "model",
      "parts": [
        "Buckshot has the following patient alerts listed in his chart:\n\n* **YELLOW - opens doors:** This likely indicates he's known to escape or be adept at opening doors/gates/kennels.  It serves as a warning to staff.\n* **ACCSR MASCOT!:**  This suggests Buckshot is a well-loved patient at the clinic, possibly a frequent visitor or just a particularly charming dog.\n* **NO AZATHIOPRINE:** This is a critical medication alert. Azathioprine should *not* be given to Buckshot.\n* **DO NOT VACCINATE:** This is another critical alert.  The reason for this alert is not explicitly stated in this portion of the record, but it's crucial that staff adhere to it.\n* **GI UPSET - careful with treats:** This indicates Buckshot has a sensitive stomach or digestive system.  Treats should be given with caution, likely limited in quantity or avoided altogether.\n\n\n",
      ],
    },
    {
      "role": "user",
      "parts": [
        "Some medications such as medicine name, strenght, number, RX description",
      ],
    },
    {
      "role": "model",
      "parts": [
        "Buckshot's medical records show a long list of medications. Here are a few examples with their strength, amount prescribed, and prescription descriptions:\n\n* **Prednisone 5mg:**  Various amounts prescribed (e.g., 30 tablets, 12 tablets) at different times.  Side effects (increased drinking, urination, and appetite) are frequently noted on the prescriptions.  Used for inflammatory and immune-mediated conditions.\n* **Metronidazole 250mg:** Prescribed in varying amounts (e.g., 100 tablets, 5 tablets). Used as an antibiotic, often for gastrointestinal issues.\n* **Metoclopramide 10mg:**  Prescribed in varying amounts (e.g., 30 tablets, 15 tablets). Instructions often indicate 1/4 tablet twice daily.  Used to treat nausea and vomiting.\n* **SiWuXiaoFeng (Si Wu Xiao Feng) 90g Powder:**  Concentrated powder, often prescribed with instructions to give 1/2 scoop mixed with food twice daily.  This is a Chinese herbal formula often used to support blood health and address various conditions.\n* **Epimedium (Epimed) 90g Powder:** Concentrated powder, also a Chinese herbal formula. Dosing instructions vary.  Used for a variety of purposes in Traditional Chinese Veterinary Medicine.\n\nThis is not an exhaustive list, as Buckshot was prescribed many other medications throughout the course of the records.  The provided information should not be interpreted as medical advice. Always consult with a veterinarian for any health concerns or before administering medication to a pet.\n",
      ],
    },
    {
      "role": "user",
      "parts": [
        "Give some Laboratory graph in chemistry lab report in marker form Date 1 to Date 6",
      ],
    },
    {
      "role": "model",
      "parts": [
        "Buckshot's records contain multiple chemistry panels. Here's a summary of some key values from six different dates:\n\n**Date 1: October 14, 2021**\n\n* **BUN:** 27 mg/dL (High)\n* **Creatinine:** 2.6 mg/dL (High)\n* **ALT (SGPT):** 62 IU/L (Within range)\n\n**Date 2: May 16, 2019**\n\n* **ALT (SGPT):** 301 IU/L (High)\n* **GGT:** 19 IU/L (High)\n* **Phosphorus:** 5.7 mg/dL (High)\n\n**Date 3: December 7, 2018**\n\n* **Phosphorus:** 6.6 mg/dL (High)\n* **Precision PSL:** 184 U/L (High)\n* **WBC:** 11.5 K/uL (High)\n\n**Date 4: February 12, 2019**\n\n* **Phosphorus:** 6.7 mg/dL (High)\n* **WBC:** 10.4 K/uL (High)\n* **ALT:** 36 U/L (Within range)\n\n**Date 5: April 24, 2020**\n\n* **BUN:** 20 mg/dL (Within range)\n* **Creatinine:** 1.4 mg/dL (Within range)\n* **ALT:** 45 IU/L (Within range)\n\n**Date 6: October 7, 2022**\n\n* **BUN:** 29 mg/dL (High)\n* **Creatinine:** 1.8 mg/dL (High)\n* **Phosphorus:** 5.0 mg/dL (Within range)\n\n\n**Key Observations:**\n\n* **Elevated Kidney Values:**  Several instances of elevated BUN and creatinine suggest potential kidney issues.\n* **Elevated Phosphorus:**  Frequently high phosphorus levels could be related to kidney problems or other metabolic imbalances.\n* **Elevated Liver Enzymes:**  High ALT and GGT on some dates indicate potential liver issues.  The Precision PSL, a marker for pancreatitis, was also elevated on one occasion.\n\n**Important Note:** This is a simplified representation of complex lab data.  A veterinarian should be consulted to interpret these results fully and determine their clinical significance.  Trends over time and other lab values not listed here are essential for accurate diagnosis and treatment.  Do not attempt to diagnose or treat a pet based on this information alone.\n",
      ],
    },
    {
      "role": "user",
      "parts": [
        "Give some Laboratory graph in CBC lab report in marker form Date 1 to Date 6",
      ],
    },
    {
      "role": "model",
      "parts": [
        "Here's a summary of some key Complete Blood Count (CBC) values from Buckshot's records across six different dates:\n\n**Date 1: October 14, 2021**\n\n* **WBC:** 6.6 K/uL (Within range)\n* **Neutrophils:** 77% (Within range)\n* **Lymphocytes:** 16% (Within range)\n\n**Date 2: May 16, 2019**\n\n* **WBC:** 6.5 K/uL (Within range)\n* **RBC:** 6.4 M/uL (Within range)\n* **HGB:** 16.1 g/dL (Within range)\n* **Neutrophils:** 81% (High)\n* **Lymphocytes:** 6% (Low)\n\n**Date 3: December 7, 2018**\n\n* **WBC:** 11.5 K/uL (High)\n* **Neutrophils:** 81% (High)\n* **Lymphocytes:** 8% (Low)\n\n**Date 4: February 12, 2019**\n\n* **WBC:** 10.4 K/uL (High)\n* **Neutrophils:** 87.4% (High)\n* **Lymphocytes:** 5.2% (Low)\n\n**Date 5: April 24, 2020**\n\n* **WBC:** 8.8 K/uL (Within range)\n* **Neutrophils:** 80% (High)\n* **Lymphocytes:** 14% (Within range)\n\n**Date 6: October 7, 2022**\n\n* **WBC:** 9.8 K/uL (Within range)\n* **Neutrophils:** 81% (High)\n* **Lymphocytes:** 10% (Low)\n\n**Key Observations:**\n\n* **Fluctuating WBC:** While generally within or slightly above the normal range, Buckshot's white blood cell count does fluctuate, suggesting an ongoing inflammatory or immune response at various times.\n* **Neutrophilia/Lymphopenia:**  The consistent pattern of high neutrophils (neutrophilia) and low lymphocytes (lymphopenia) is a common finding in dogs with stress, inflammation, or certain infections.\n\n**Important Note:**  This is a simplified overview, and a veterinarian should be consulted for a complete interpretation.  Trends, other CBC parameters not listed, and the clinical picture are crucial for accurate diagnosis and treatment.  Do not attempt to self-diagnose or treat a pet based on this information.\n",
      ],
    },
    {
      "role": "user",
      "parts": [
        "Give some Laboratory graph in Endocrinology lab report in marker form Date 1 to Date 6",
      ],
    },
    {
      "role": "model",
      "parts": [
        "Buckshot's records include several T4 (thyroxine) measurements, which are part of an endocrine panel assessing thyroid function. Here are the T4 values from six different dates:\n\n* **May 16, 2019:** 2.0 μg/dL (Within range)\n* **December 7, 2018:** 1.5 μg/dL (Within range)\n* **February 12, 2019:** Not explicitly listed on this date's chemistry panel.\n* **April 24, 2020:** Not explicitly listed on this date's chemistry panel.\n* **October 7, 2022:** 3.0 μg/dL (Within range)\n* **May 8, 2024:** 1.1 μg/dL (Within range)\n\n**Key Observations:**\n\nAll of Buckshot's recorded T4 values fall within the normal reference range (typically 0.8-3.5 μg/dL). This suggests that his thyroid function was generally considered normal at the times these tests were performed.\n\n**Important Note:**  While these T4 values are within the normal range, a single T4 measurement doesn't always provide a complete picture of thyroid health.  Other thyroid tests, such as free T4 and TSH (thyroid-stimulating hormone), might be necessary for a more thorough evaluation, especially if there are clinical signs of hypothyroidism or hyperthyroidism.  Always consult with a veterinarian to interpret thyroid test results and determine the appropriate course of action.  Do not attempt to diagnose or treat thyroid conditions based on this information alone.\n",
      ],
    },
    {
      "role": "user",
      "parts": [
        "Give some Laboratory graph in Urinalysis lab report in marker form Date 1 to Date 6",
      ],
    },
    {
      "role": "model",
      "parts": [
        "Buckshot's records contain several urinalysis reports. Here's a summary of key findings from six different dates:\n\n**Date 1: October 7, 2022**\n\n* **Color:** Dark Yellow\n* **Specific Gravity:** 1.046 (Normal to slightly concentrated)\n* **Protein:** Trace (May warrant further investigation)\n\n**Date 2: May 16, 2019**\n\n* **Color:** Dark Yellow\n* **Appearance:** Cloudy\n* **Specific Gravity:** 1.050 (Normal to slightly concentrated)\n* **Protein:** 1+ (High - suggests proteinuria)\n* **Fat Droplets:** 11-20 HPF (High)\n\n**Date 3: December 7, 2018**\n\n* **Color:** Dark Yellow\n* **Specific Gravity:** 1.049 (Normal to slightly concentrated)\n* **Protein:** Trace (May warrant further investigation)\n\n**Date 4: February 12, 2019** (Urinalysis not explicitly listed on this date)\n\n**Date 5: April 24, 2020** (Urinalysis not explicitly listed on this date)\n\n**Date 6: May 8, 2024**\n\n* **Color:** Yellow\n* **Specific Gravity:** 1.015 (Normal)\n* **Protein:** Negative\n* **WBC:** 0-3 hpf (Normal)\n* **RBC:** None seen hpf (Normal)\n\n**Key Observations:**\n\n* **Proteinuria:** The presence of protein in the urine (especially the 1+ reading on May 16, 2019) is a significant finding and warrants further investigation.  Protein in the urine can be a sign of kidney disease, urinary tract infection, or other conditions.\n* **Specific Gravity:**  The specific gravity readings are mostly within the normal range, indicating adequate kidney function in concentrating urine.\n* **Fat Droplets:** The high number of fat droplets on May 16, 2019, could be related to various factors and should be investigated further.\n\n**Important Note:** This is a simplified overview, and a full interpretation requires considering all urinalysis parameters, along with the clinical picture and other diagnostic tests.  Always consult with a veterinarian to interpret urinalysis results and determine the appropriate course of action.  Do not attempt to self-diagnose or treat a pet based on this information.\n",
      ],
    },
    {
      "role": "user",
      "parts": [
        "Give some Laboratory graph in Heartworm test lab report in marker form Date 1 to Date 6",
      ],
    },
    {
      "role": "model",
      "parts": [
        "Buckshot's records show several heartworm tests, all with negative results:\n\n* **October 14, 2021:** Negative\n* **October 7, 2022:** Negative\n* **May 16, 2019:** Negative\n* **May 7, 2025:** Scheduled (future test, result not yet available)\n* **May 8, 2024:** Negative\n* **May 7, 2024:** Negative\n\n**Key Observation:**\n\nAll recorded and scheduled heartworm tests for Buckshot are negative, indicating he has not been infected with heartworms at the times of testing.\n\n**Important Note:**  Negative heartworm tests are good news, but it's crucial to maintain regular heartworm prevention, especially in areas where heartworm disease is prevalent.  Heartworm prevention medication is highly effective in protecting dogs from this potentially life-threatening parasite.  Consult with a veterinarian about the best heartworm prevention strategy for Buckshot.  Do not attempt to diagnose or treat heartworm disease based on this information alone.\n",
      ],
    },
    {
      "role": "user",
      "parts": [
        "Give some Laboratory graph in Fecal results lab report in marker form Date 1 to Date 6",
      ],
    },
    {
      "role": "model",
      "parts": [
        "Buckshot's records include several fecal examinations. Here's a summary of the findings from six different dates:\n\n* **May 22, 2021:** No parasites seen.\n* **May 17, 2018:** No parasites seen.\n* **May 5, 2018:** Negative for intestinal parasites.\n* **May 7, 2025:** Scheduled (future test, result not yet available)\n* **October 6, 2022:**  (Fecal scheduled but results not in this document)\n* **May 8, 2024:** No ova or parasites seen.\n\n**Key Observation:**\n\nAll recorded fecal examinations for Buckshot have been negative for parasites.\n\n**Important Note:**  While these negative results are positive, it's important to remember that fecal tests provide a snapshot in time.  A negative result doesn't guarantee a pet is completely parasite-free, as some parasites can be intermittent shedders.  Regular fecal testing and appropriate parasite prevention measures are essential for maintaining a pet's health.  Consult with a veterinarian about the best parasite prevention protocol for Buckshot.  Do not attempt to diagnose or treat parasitic infections based on this information alone.\n",
      ],
    },
    {
      "role": "user",
      "parts": [
        "Give some Laboratory graph in other results lab report in marker form Date 1 to Date 6",
      ],
    },
    {
      "role": "model",
      "parts": [
        "Buckshot's records include a few other lab results beyond the standard CBC, Chemistry, Urinalysis, Fecal, and Heartworm tests. Here are some examples:\n\n1. **May 21, 2019 (Corneal Ulcer Culture):** Moderate growth of Gram-negative organisms, specifically *Escherichia coli*, was found.  This indicates a bacterial infection of the cornea.  A sensitivity test was performed to determine the appropriate antibiotic treatment.\n\n2. **May 7, 2019 (Schirmer Tear Test):**  This test measures tear production.  The actual numerical result isn't provided in this section of the record, but it was performed in response to Buckshot's recurring conjunctivitis.\n\n3. **December 7, 2018 (Canine C-reactive Protein):** This test measures inflammation in the body. The specific result isn't provided in this part of the record.\n\n4. **May 28, 2019 (Aerobic Culture and Sensitivity from right ear - not shown):**  While the results aren't in this document section, the entry indicates an ear infection and the need for a culture to identify the bacteria and determine appropriate antibiotic treatment.\n\n5. **August 18, 2019 (Ear Cytology - not shown):**  Another ear cytology was performed, but the results are not included in this section.\n\n6. **September 18, 2024 (Animal Biome Gut Microbiome Report):**  This report likely provided detailed information about the bacteria present in Buckshot's gut.  The specific findings are not included here.\n\n**Important Note:**  These are just a few examples, and there may be other lab results scattered throughout the full medical record.  It's essential to review the complete record with a veterinarian for a comprehensive understanding of Buckshot's health history.  Do not attempt to interpret or act on these isolated lab results without professional guidance.\n",
      ],
    },
    {
      "role": "user",
      "parts": [
        "Give Imaging of radio graphs of date, position and results",
      ],
    },
    {
      "role": "model",
      "parts": [
        "Radiographs were taken of Buckshot on **January 23, 2023**.  The record indicates the images focused on the **cervical spine (neck)**. The findings describe \"hemivertebrae and butterfly vertebrae thoracic. varying disc spaces from C5-C7.\"\n\n* **Hemivertebrae:** A congenital malformation where only half of a vertebra develops.\n* **Butterfly Vertebrae:** Another congenital malformation where the vertebra has a cleft through the middle, resembling a butterfly.\n* **Varying disc spaces from C5-C7:** This refers to irregularities in the spacing between the vertebral discs in the lower neck region, which could indicate disc disease or other abnormalities.\n\nNo other radiograph reports are included in the provided document.  It's important to note that the \"position\" (e.g., lateral, ventrodorsal) of the radiographs isn't explicitly stated in this entry.  A veterinarian would need to review the actual images for a complete interpretation.\n\n\n",
      ],
    },
    {
      "role": "user",
      "parts": [
        "Give Imaging of ultrasound of date, position and results",
      ],
    },
    {
      "role": "model",
      "parts": [
        "An ultrasound was performed on Buckshot on **February 27, 2023**. The ultrasound focused on the **jejunum**, a part of the small intestine. The findings describe the \"mucosa [as] striated.\"  This finding can be associated with inflammatory bowel disease (IBD) or other gastrointestinal conditions.\n\nNo other ultrasound reports are included in the provided document.  It's important to note that the \"position\" of the ultrasound probe isn't explicitly stated in this entry.  A veterinarian would need to review the actual ultrasound images and the complete medical record for a full interpretation.\n",
      ],
    },
  ]
)

response = chat_session.send_message("Give structured Document for above chats only")

print(response.text)

## Buckshot's Medical Record Summary

**Client Information:**

* **Name:** Teleah Grand
* **Phone:** 214-563-1688
* **Address:** 2200 Victory Ave #702, Dallas, TX 75219

**Patient Information:**

* **Name:** Buckshot
* **Breed:** French Bulldog
* **Date of Birth:** February 28, 2018 (6 years old at the time of the last record entry)
* **Gender:** Neutered Male
* **Microchip:** 981020021462501

**Veterinary Clinic:**

* **Name:** Animal Care Center at Stonebridge Ranch
* **Phone:** 214-563-1688
* **Record Dates:** May 5, 2018 - October 14, 2024

**Patient Alerts:**

* YELLOW - opens doors
* ACCSR MASCOT!
* NO AZATHIOPRINE
* DO NOT VACCINATE
* GI UPSET - careful with treats

**Vaccination History:**

* **Rabies:** June 1, 2018 (Merial; Tag #432943) - Expired June 22, 2019; Titer performed November 30, 2020 (results pending).
* **DA2PP:** Multiple administrations (see original response for dates and manufacturers).
* **Bordetella:** Multiple administrations (see original response for date